## Example 14 - Emission footprint of IWT

#### Create an environment named: "gdal"

Install Anaconda3-2020.07-Windows-x86_64.exe

In the Anaconda prompt execute the following commands:

    conda create --name gdal
    conda activate gdal

    conda config --add channels conda-forge 
    conda config --set channel_priority strict

    conda install gdal
    yes
    conda install geopandas
    yes
    conda install folium
    yes
    conda install pyyaml
    yes

    cd c:\Users\MRV\OneDrive - Van Oord\Software\github\OpenTNSim\
    pip install -e .

    conda install jupyter
    yes

    cd ..
    jupyter notebook

https://nbviewer.jupyter.org/github/Deltares/digitaltwin-waterway/blob/master/notebooks/scenario-discharge.ipynb

In [21]:
import opentnsim
print('This notebook has been tested with OpenTNSim version {} - branch: Afstuderen_LoesSegers'.format(opentnsim.__version__))

This notebook has been tested with OpenTNSim version 1.0.0 - branch: Afstuderen_LoesSegers


In [22]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib
import io
import urllib
import tempfile
import functools
import logging
import pickle

# package(s) related to the simulation
import simpy
import networkx as nx  

# OpenTNSIM
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot
import opentnsim.model as model

# spatial libraries 
import shapely.geometry
import shapely.wkt
import pyproj
import shapely.geometry
from osgeo import ogr, osr
from simplekml import Kml, Style
import folium

# package(s) for data handling
import requests
import math             
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# define the coorinate system
geod = pyproj.Geod(ellps="WGS84")

In [23]:
# Link to the latest version of the 'Vaarweginformatie.nl' network
url = 'https://zenodo.org/record/3981105/files/network_digital_twin_v0.1.yaml'

In [24]:
# Link to vessel database
location_vessel_database = "Vessels\Vessel-database-2.csv"

### 0. Load useful methods

In [25]:
@functools.lru_cache
def load_fis_network(url):
    """load the topological fairway information system network (vaarweginformatie.nl)"""

    # get the data from the url
    resp = requests.get(url)
    # convert to file object
    stream = io.StringIO(resp.text)
    
    # This will take a minute or two
    # Here we convert the network to a networkx object
    G = nx.read_yaml(stream)

    # some brief info
    n_bytes = len(resp.content)
    msg = '''Loaded network from {url} file size {mb:.2f}MB. Network has {n_nodes} nodes and {n_edges} edges.'''
    summary = msg.format(url=url, mb=n_bytes / 1000**2, n_edges=len(G.edges), n_nodes=len(G.nodes))
    logger.info(summary)

    # The topological network contains information about the original geometry. 
    # Let's convert those into python shapely objects for easier use later
    for n in G.nodes:
        G.nodes[n]['geometry'] = shapely.geometry.Point(G.nodes[n]['X'], G.nodes[n]['Y'])
    for e in G.edges:
        edge = G.edges[e]
        edge['geometry'] = shapely.wkt.loads(edge['Wkt'])
        edge['length'] = edge_length(edge)    
    
    return G 

In [26]:
def find_closest_node(G, point):
    """find the closest node on the graph from a given point"""
    
    distance = np.full((len(G.nodes)), fill_value=np.nan)
    for ii, n in enumerate(G.nodes):
        distance[ii] = point.distance(G.nodes[n]['geometry'])
    name_node = list(G.nodes)[np.argmin(distance)]
    distance_node = np.min(distance)
    
    return name_node, distance_node

In [27]:
def find_closest_edge(G, point):
    """find the closest edge on the graph from a given point"""
    
    distance = np.full((len(G.edges)), fill_value=np.nan)
    for ii, e in enumerate(G.edges):
        distance[ii] = point.distance(G.edges[e]['geometry'])
    name_edge = list(G.edges)[np.argmin(distance)]
    distance_edge = np.min(distance)
    
    return name_edge, distance_edge

In [28]:
def edge_length(edge):
    """compute the great circle length of an edge
    The network version 0.1 contains the lat/lon distance in a length property. 
    But we need the "great circle" or projected distance. 
    Let's define a function to recompute it.
    """
    
    # get the geometry
    geom = edge['geometry']
    # get lon, lat
    lats, lons = np.array(geom).T
    # this requires pyproj 2.3.0
    distance = geod.line_length(lons, lats)

    return distance

In [29]:
class VesselProperties:
    """Mixin class: Something that has vessel properties
    This mixin is updated to better accommodate the ConsumesEnergy mixin

    type: can contain info on vessel type (avv class, cemt_class or other)
    B: vessel width
    L: vessel length
    H_e: vessel height unloaded
    H_f: vessel height loaded
    T_e: draught unloaded
    T_f: draught loaded

    Add information on possible restrictions to the vessels, i.e. height, width, etc.
    """

    def __init__(
        self,
        type,
        B,
        L,
        H_e,
        H_f,
        T_e,
        T_f,
        *args,
        **kwargs
        ):
        super().__init__(*args, **kwargs)

        """Initialization"""
        self.type = type

        self.B = B
        self.L = L

        self.H_e = H_e
        self.H_f = H_e

        self.T_e = T_e
        self.T_f = T_f


        
    @property
    def H(self):
        """ Calculate current height based on filling degree """

        return (
            self.filling_degree * (self.H_f - self.H_e)
            + self.H_e
        )

    @property
    def T(self):
        """ Calculate current draught based on filling degree
        
        Here we should implement the rules from Van Dorsser et al 
        https://www.researchgate.net/publication/344340126_The_effect_of_low_water_on_loading_capacity_of_inland_ships
        """

        return (
            self.filling_degree * (self.T_f - self.T_e)
            + self.T_e
        )

    def get_route(
        self,
        origin,
        destination,
        graph=None,
        minWidth=None,
        minHeight=None,
        minDepth=None,
        randomSeed=4,
    ):
        """ Calculate a path based on vessel restrictions """

        graph = graph if graph else self.env.FG
        minWidth = minWidth if minWidth else 1.1 * self.B
        minHeight = minWidth if minHeight else 1.1 * self.H
        minDepth = minWidth if minDepth else 1.1 * self.T

        # Check if information on restrictions is added to the edges
        random.seed(randomSeed)
        edge = random.choice(list(graph.edges(data=True)))
        edge_attrs = list(edge[2].keys())

        # IMPROVE THIS TO CHECK ALL EDGES AND COMBINATIONS OF RESTRICTIONS

        if all(item in edge_attrs for item in ["Width", "Height", "Depth"]):
            edges = []
            nodes = []

            for edge in graph.edges(data=True):
                if (
                    edge[2]["Width"] >= minWidth
                    and edge[2]["Height"] >= minHeight
                    and edge[2]["Depth"] >= minDepth
                ):
                    edges.append(edge)

                    nodes.append(graph.nodes[edge[0]])
                    nodes.append(graph.nodes[edge[1]])

            subGraph = graph.__class__()

            for node in nodes:
                subGraph.add_node(
                    node["name"],
                    name=node["name"],
                    geometry=node["geometry"],
                    position=(node["geometry"].x, node["geometry"].y),
                )

            for edge in edges:
                subGraph.add_edge(edge[0], edge[1], attr_dict=edge[2])

            try:
                return nx.dijkstra_path(subGraph, origin, destination)
            except:
                raise ValueError(
                    "No path was found with the given boundary conditions."
                )

        # If not, return shortest path
        else:
            return nx.dijkstra_path(graph, origin, destination)


In [30]:
# # Define a EnergyCalculation class (this class post processes all results and calculates the energy (kWh))
# class EnergyCalculation:
#     """
#     Add information on energy use and effects on energy use.
#     """

#     def __init__(self, vessel, *args, **kwargs):
#         super().__init__(*args, **kwargs)

#         """Initialization"""
#         self.vessel = vessel
        
#         self.energy_use = {"time_start": [], 
#                            "time_stop": [], 
#                            "edge_start": [],
#                            "edge_stop": [],
#                            "total_energy": [], 
#                            "stationary": []}
        
#         self.co2_footprint = {"total_footprint": 0, "stationary": 0} # to be filled
#         self.mki_footprint = {"total_footprint": 0, "stationary": 0} # to be filled

#     def calculate_energy_consumption(self):
#         """Calculation of energy consumption based on total time in system and properties"""

#         def calculate_distance(geom_start, geom_stop):
#             """method to calculate the distance in meters between two geometries"""
#             wgs84 = pyproj.Geod(ellps='WGS84')

#             # distance between two points
#             return float(wgs84.inv(geom_start.x, geom_start.y,
#                                  geom_stop.x,  geom_stop.y) [2])
        
#         # log messages that are related to locking
#         stationary_phase_indicator = [
#             "Waiting to enter waiting area stop",
#             "Waiting in waiting area stop",
#             "Waiting in line-up area stop",
#             "Passing lock stop",
#         ]
        
#         # extract relevant elements from the vessel log
#         times = self.vessel.log["Timestamp"]
#         messages = self.vessel.log["Message"]
#         geometries = self.vessel.log["Geometry"]

#         # now walk past each logged event in vessel.log (each 'time interval' in the log corresponds to an event)
#         for i in range(len(times) - 1):
#             # determine the time associated with the logged event (how long did it last [seconds])
#             delta_t = (times[i + 1] - times[i]).total_seconds()
            
#             if delta_t != 0: # filters out intervals between previous stop and next start (should the same)
#                 # append time information to the variables for the dataframe
#                 self.energy_use["time_start"].append(times[i])
#                 self.energy_use["time_stop"].append(times[i + 1])
                
#                 # append geometry information to the variables for the dataframe
#                 self.energy_use["edge_start"].append(geometries[i])
#                 self.energy_use["edge_stop"].append(geometries[i + 1])
                
#                 # calculate the distance travelled and the associated velocity 
#                 distance = calculate_distance(geometries[i], geometries[i + 1])
#                 V_0 = distance / delta_t

#                 # printstatements to check the output (can be removed later)
#                 print('delta_t: {:.4f} s'. format(delta_t))
#                 print('distance: {:.4f} m'. format(distance))
#                 print('velocity: {:.4f} m/s'. format(V_0))
                
#                 # we use the calculated velocity to determine the resistance (at h=10 for now) and power required
#                 self.vessel.calculate_total_resistance(V_0, h=10)
#                 self.vessel.calculate_total_power_required()

#                 if messages[i + 1] in stationary_phase_indicator:  # if we are in a stationary phase only log P_hotel
#                     energy_delta = self.vessel.P_hotel * delta_t / 3600  # kJ/3600 = kWh
#                     self.energy_use["total_energy"].append(energy_delta)
#                     self.energy_use["stationary"].append(energy_delta)

#                 else:  # otherwise log P_tot
#                     energy_delta = self.vessel.P_tot * delta_t / 3600  # kJ/3600 = kWh
#                     self.energy_use["total_energy"].append(energy_delta)
#                     self.energy_use["stationary"].append(0)
        
#         # TODO: er moet hier een heel aantal dingen beter worden ingevuld
#         # - de kruissnelheid is nu nog per default 1 m/s (zie de Movable mixin). Eigenlijk moet in de 
#         #   vessel database ook nog een speed_loaded en een speed_unloaded worden toegevoegd. 
#         # - er zou nog eens goed gekeken moeten worden wat er gedaan kan worden rond kustwerken 
#         # - en er is nog iets mis met de snelheid rond een sluis

#     def plot(self):
        
#         import folium

#         df = pd.DataFrame.from_dict(energycalculation.energy_use)

#         m = folium.Map(location=[51.7, 4.4], zoom_start = 12)
        
#         line = []
#         row = df[df.index==0]
#         line.append((row["edge_start"][0].y, row["edge_start"][0].x))
#         folium.Marker([row["edge_start"][0].y, row["edge_start"][0].x], 
#                       icon=folium.Icon(color='red'),
#                       popup='Route start').add_to(m)
#         for index, row in df.iterrows():
#             line.append((row["edge_stop"].y, row["edge_stop"].x))
#             folium.Marker([row["edge_stop"].y, row["edge_stop"].x]).add_to(m)

#         folium.PolyLine(line, weight = 4).add_to(m)

#         return m

In [31]:
# This mixin now contains the calculation rules as proposed by Loes Segers
# NB: all calculate values are added as a property to the vessel object (not sure this is needed/wise)
#     alternatively we could work more with inputs and returns to just get back the value you need

class ConsumesEnergy:
    """Mixin class: Something that consumes energy.

    P_installed: installed engine power [kW]
    L_w: weight class of the ship (depending on carrying capacity) (classes: L1 (=1), L2 (=2), L3 (=3))
    C_b: block coefficient ('fullness') [-]
    C_BB: breadth coefficient (determines area of bulbous bow) [-]
    nu: kinematic viscosity [m^2/s]
    rho: density of the surrounding water [kg/m^3]
    g: gravitational accelleration [m/s^2]
    x: number of propellors [-]
    eta_0: open water efficiency of propellor [-]
    eta_r: relative rotative efficiency [-]
    eta_t: transmission efficiency [-]
    eta_g: gearing efficiency [-]
    c_stern: determines shape of the afterbody [-]
    one_k2: appendage resistance factor [-]
    """

    def __init__(
        self, 
        P_installed,
        L_w,
        C_b,   
        C_BB,
        nu = 1*10**(-6), # kinematic viscosity
        rho = 1000,
        g = 9.81,
        x = 2, # number of propellors
        eta_0 = 0.7,
        eta_r = 0.98,
        eta_t = 0.98,
        eta_g = 0.96,
        c_stern = 0,
        one_k2 = 1.5,
        *args, 
        **kwargs
        ):
        super().__init__(*args, **kwargs)

        """Initialization"""
        self.P_installed = P_installed
        self.L_w = L_w
        self.C_b = C_b  
        self.C_BB = C_BB
        self.nu = nu 
        self.rho = rho
        self.g = g
        self.x = x
        self.eta_0 = eta_0
        self.eta_r = eta_r
        self.eta_t = eta_t
        self.eta_g = eta_g
        self.c_stern = c_stern
        self.one_k2 = one_k2
        self.c_year = self.calculate_engine_age()   #The construction year of the engine is now generated once, instead of for each time step

# The engine age and construction year of the engine is computed with the function below.
# The construction year of the engine is used in the emission functions (1) emission_factors_general and (2) correction_factors

    def calculate_engine_age(self):
        """Calculating the construction year of the engine, dependend on a Weibull function with 
        shape factor 'k', and scale factor 'lmb', which are determined by the weight class L_w"""
        
        #Determining which shape and scale factor to use, based on the weight class L_w = L1, L2 or L3
        if self.L_w == 1:     #Weight class L1
            self.k = 1.3
            self.lmb = 20.5
        if self.L_w == 2:     #Weight class L2
            self.k = 1.12
            self.lmb = 18.5
        if self.L_w == 3:     #Weight class L3
            self.k = 1.26
            self.lmb = 18.6
            
        #The age of the engine
        self.age = int(np.random.weibull(self.k)*self.lmb)
        
        #Current year (TO DO: fix hardcoded year)
        #self.year = datetime.date.year
        self.year = 2020
        
        #Construction year of the engine
        self.c_year = self.year - self.age
        
        print('The construction year of the engine is', self.c_year)
        return self.c_year
            
    def calculate_properties(self):
        """Calculate a number of basic vessel properties"""
        self.C_M = 1.006 - 0.0056 * self.C_b ** (-3.56)          #Midship section coefficient
        self.C_wp = (1 + 2 * self.C_b) / 3                       #Waterplane coefficient
        self.C_p = self.C_b / self.C_M                           #Prismatic coefficient

        self.delta = self.C_b * self.L * self.B * self.T         #Water displacement

        self.lcb = -13.5 + 19.4 * self.C_p                                                     #longitudinal center of buoyancy
        self.L_R = self.L * (1 - self.C_p + (0.06 * self.C_p * self.lcb)/(4 * self.C_p - 1))   #parameter reflecting the length of the run

        self.A_BT = self.C_BB * self.B * self.T * self.C_M       #wet transverse sectional area of the bulbous bow
        self.A_T = 0.1 * self.B * self.T                         #transverse area of the transom
        
        #Total wet area
        self.S_T = self.L * (2 * self.T + self.B) * np.sqrt(self.C_M) * (0.453 + 0.4425 * self.C_b - 0.2862 * self.C_M - 0.003467 * (self.B / self.T) + 0.3696 * self.C_wp) + 2.38 * (self.A_BT / self.C_b)
       
        self.S_APP = 0.05 * self.S_T       #Wet area of appendages
        self.S_B = self.L * self.B         #Area of flat bottom

        self.D_s = 0.7 * self.T            #Diameter of the screw
        
        
    def calculate_frictional_resistance(self, V_0, h):
        """Section 2.2.1 Frictional resistance"""

        self.R_e = V_0 * self.L / self.nu        #Reynolds number
        self.D = h - self.T                      #distance from bottom ship to the bottom of the fairway
        
        #Friction coefficient in deep water
        self.Cf_0 = 0.075 / ((np.log10(self.R_e) - 2) ** 2)
        
        #Friction coefficient proposed, taking into account shallow water effects
        self.Cf_proposed = (0.08169 / ((np.log10(self.R_e) - 1.717) ** 2)) * (1 + (0.003998 / (np.log10(self.R_e) - 4.393)) * (self.D / self.L) ** (-1.083))

        #'a' is the coefficient needed to calculate the Katsui friction coefficient
        self.a = 0.042612 * np.log10(self.R_e) + 0.56725
        self.Cf_katsui = 0.0066577 / ((np.log10(self.R_e) - 4.3762) ** self.a)

        #The average velocity underneath the ship, taking into account the shallow water effect
        self.V_B = 0.4277 * V_0 * np.exp((h / self.T) ** (-0.07625))
        
        #cf_proposed cannot be applied directly, since a vessel also has non-horizontal wet surfaces that have to be taken
        #into account. Therefore, the following formula for the final friction coefficient 'C_f' is defined:
        self.C_f = self.Cf_0 + (self.Cf_proposed - self.Cf_katsui) * (self.S_B / self.S_T) * (self.V_B / V_0) ** 2

        #The total frictional resistance R_f [kN]:
        self.R_f = (self.C_f * 0.5 * self.rho * (V_0 ** 2) * self.S_T) / 1000 

        
    def calculate_viscous_resistance(self):
        """Section 2.2.2 Viscous resistance"""
        
        #c_14 accounts for the specific shape of the afterbody
        self.c_14 = 1 + 0.0011 * self.c_stern
        
        #the form factor (1+k1) describes the viscous resistance
        self.one_k1 = 0.93 + 0.487 * self.c_14 * ((self.B / self.L) ** 1.068) * ((self.T/self.L) ** 0.461) * ((self.L / self.L_R) ** 0.122) * (((self.L ** 3) / self.delta) ** 0.365) * ((1 - self.C_p) ** (-0.604))
        
    def calculate_appendage_resistance(self, V_0):
        """Section 2.2.3 Appendage resistance"""
        
        #Resistance resulting from wet area of appendages: R_APP [kN]
        self.R_APP = (0.5 * self.rho * (V_0 ** 2) * self.S_APP * self.one_k2 * self.C_f) / 1000
     
    
    def calculate_wave_resistance(self, V_0):
        """Section 2.2.4 Wave resistance"""

        self.F_n = V_0 / np.sqrt(self.g * self.L)          #Froude number

        #coefficient c_7
        if self.B / self.L < 0.11:
            self.c_7 = 0.229577 * (self.B / self.L) ** 0.33333
        elif self.B / self.L > 0.25:
            self.c_7 = 0.5 - 0.0625 * (self.L / self.B)
        else:
            self.c_7 = self.B / self.L
       
        #Half angle of entrance i_E
        self.i_E = 125.67 * (self.B / self.L) - 162.25 * (self.C_p ** 2) + 234.32 * (self.C_p ** 3) + 0.155087 * (self.lcb ** 3)
        self.c_1 = 2223105 * (self.c_7 ** 3.78613) * ((self.T / self.B) ** 1.07961) * ((90 - self.i_E) ** (-1.37565))

        #Parameter c_2 (depending on c_3) accounts for the reduction of wave resistance due to the action of the bulbous bow
        self.c_3 = 0.56 * (self.A_BT ** 1.5) / (self.B * self.T * (0.31 * np.sqrt(self.A_BT) + self.T - 0.5*self.T))
        self.c_2 = np.exp(-1.89 * np.sqrt(self.c_3))
        
        #c_5 accounts for the effect of the transom stern on the wave resistance.
        self.c_5 = 1 - 0.8 * self.A_T / (self.B * self.T * self.C_M)

        #coefficient c_16
        if self.C_p < 0.80:
            self.c_16 = 8.07981 * self.C_p - (13.8673 * self.C_p ** 2) + (6.984388 * self.C_p ** 3)
        else:
            self.c_16 = 1.73014 - 0.7067 * self.C_p

        self.m_1 = 0.0140407 * (self.L/self.T) - (1.75254 * self.delta**(1/3))/self.L - 4.79323 * (self.B/self.L) - self.c_16

        #coefficient c_15
        if (self.L**3)/self.delta < 512:
            self.c_15 = -1.69385
        elif (self.L**3)/self.delta > 1727:
            self.c_15 = 0
        else:
            self.c_15 = -1.69385 + (self.L / (self.delta**(1/3)) - 8.0) / 2.36

        self.m_2 = self.c_15 * (self.C_p**2) * np.exp(-0.1 * (self.F_n)**(-2))

        #coefficient lambda
        if self.L/self.B < 12:
            self.lmbda = 1.446 * self.C_p - 0.03 * (self.L/self.B)
        else:
            self.lmbda = 1.446 * self.C_p - 0.36

        #The wave resistance depends on all coefficients above: R_W [kN] 
        self.R_W = (self.c_1 * self.c_2 * self.c_5 * self.delta * self.rho * self.g * np.exp(self.m_1 * (self.F_n**-0.9) + self.m_2 * np.cos(self.lmbda * (self.F_n**(-2))))) / 1000 

    def calculate_residual_resistance(self, V_0):
        """Section 2.2.5 Residual resistance terms"""

        #Resistance resulting from the bulbouw bow: R_B [kN]
        self.P_B = 0.56 * np.sqrt(self.A_BT) / (self.T - 1.5 * 0.5*self.T)     #measure for emergence of bulbouw bow
        self.F_ni = V_0 / np.sqrt(self.g * (self.T - 0.5*self.T - 0.25*np.sqrt(self.A_BT)) + 0.15*(V_0**2))    #Froude number based on immersion

        self.R_B = (0.11 * np.exp(-3 * self.P_B**(-2)) * (self.F_ni**3) * (self.A_BT**1.5) * self.rho * self.g / (1 + (self.F_ni**2))) / 1000

        #Resistance due to immersed transom: R_TR [kN]
        self.F_nt = V_0 / np.sqrt(2 * self.g * self.A_T / (self.B + self.B * self.C_wp))    #Froude number based on transom immersion
        self.c_6 = 0.2 * (1 - 0.2 * self.F_nt)      #Assuming F_nt < 5, this is the expression for coefficient c_6

        self.R_TR = (0.5 * self.rho * (V_0**2) * self.A_T * self.c_6) / 1000 

        #Model-ship correlation resistance: R_A [kN]
        if self.T/self.L > 0.04:
            self.c_4 = 0.04
        else:
            self.c_4 = self.T / self.L

        self.C_A = 0.006 * ((self.L + 100) ** (-0.16)) - 0.00205 + 0.003 * np.sqrt(self.L / 7.5) * (self.C_b ** 4) * self.c_2 * (0.04 - self.c_4)

        self.R_A = (0.5 * self.rho * (V_0 ** 2) * self.S_T * self.C_A) / 1000 #kW

    def calculate_total_resistance(self, V_0, h):
        """Calculating the total resistance (which is the sum of all the resistance components)"""
        
        self.calculate_properties()
        self.calculate_frictional_resistance(V_0, h)
        self.calculate_viscous_resistance()
        self.calculate_appendage_resistance(V_0)
        self.calculate_wave_resistance(V_0)
        self.calculate_residual_resistance(V_0)
        
        #The total resistance R_tot [kN] = R_f * (1+k1) + R_APP + R_W + R_B + R_TR + R_A
        self.R_tot = self.R_f * self.one_k1 + self.R_APP + self.R_W + self.R_B + self.R_TR + self.R_A

    def calculate_total_power_required(self):
        """ Section 2.1 Total required power """

        #2.1.1 Required power for systems on board
        self.P_hotel = 0.081 * self.P_installed

        #2.1.2 Required power for propulsion

        #Effective Horse Power (EHP)
        self.P_EHP = self.V_B * self.R_tot
        
        #Calculation hull efficiency
        dw = np.zeros(101)              #velocity correction coefficient
        counter = 0 
        
        if self.F_n < 0.2:
            self.dw = 0
        else:
            self.dw = 0.1


        self.w = 0.11 * (0.16 / self.x) * self.C_b * np.sqrt((self.delta**(1/3)) / self.D_s) - self.dw   #wake fraction 'w'


        if self.x == 1:
            self.t = 0.6 * self.w * (1 + 0.67 * self.w)       #thrust deduction factor 't'
        else:
            self.t = 0.8 * self.w * (1 + 0.25 * self.w)

        self.eta_h = (1 - self.t) / (1 - self.w)    #hull efficiency eta_h

        #Delivered Horse Power (DHP)

        self.P_DHP = self.P_EHP / (self.eta_0 * self.eta_r * self.eta_h)

        #Brake Horse Power (BHP)
        self.P_BHP = self.P_DHP / (self.eta_t * self.eta_g)

        self.P_tot = self.P_hotel + self.P_BHP
        
        #Partial engine load (P_partial): needed in the emission calculations (correction factors depend on P_partial)
        self.P_partial = self.P_tot / self.P_installed
        
        print('The total power required is', self.P_tot, 'kW')
        print('The partial load is', self.P_partial, 'kW')


        
        
    def emission_factors_general(self):
        """Calculating the general emission factors, based on construction year of the engine.
        
        Please note: later on a correction factor has to be applied to get the total emission factor"""
        
        #The general emission factors of CO2, PM10 and NOX are based on the construction year of the engine
        
        if self.c_year < 1974:
            self.EM_CO2 = 756
            self.EM_PM10 = 0.6
            self.EM_NOX = 10.8
        if 1975 <= self.c_year <= 1979:
            self.EM_CO2 = 730
            self.EM_PM10 = 0.6
            self.EM_NOX = 10.6
        if 1980 <= self.c_year <= 1984:
            self.EM_CO2 = 714
            self.EM_PM10 = 0.6
            self.EM_NOX = 10.4
        if 1985 <= self.c_year <= 1989:
            self.EM_CO2 = 698
            self.EM_PM10 = 0.5
            self.EM_NOX = 10.1
        if 1990 <= self.c_year <= 1994:
            self.EM_CO2 = 698
            self.EM_PM10 = 0.4
            self.EM_NOX = 10.1
        if 1995 <= self.c_year <= 2002:
            self.EM_CO2 = 650
            self.EM_PM10 = 0.3
            self.EM_NOX = 9.4
        if 2003 <= self.c_year <= 2007:
            self.EM_CO2 = 635
            self.EM_PM10 = 0.3
            self.EM_NOX = 9.2
        if 2008 <= self.c_year <= 2019:
            self.EM_CO2 = 635
            self.EM_PM10 = 0.2
            self.EM_NOX = 7
        if self.c_year > 2019:
            if self.L_w == 1:
                self.EM_CO2 = 650
                self.EM_PM10 = 0.1
                self.EM_NOX = 2.9
            else:
                self.EM_CO2 = 603
                self.EM_PM10 = 0.0015
                self.EM_NOX = 2.4
                
        print('The general emission factor of CO2 is', self.EM_CO2, 'g/kWh')
        print('The general emission factor of PM10 is', self.EM_PM10, 'g/kWh')
        print('The general emission factor CO2 is', self.EM_NOX, 'g/kWh')      
        
    def correction_factors(self):
        """Calculating the correction factors that have to be applied to the general emission factors, 
        based on the partial engine load of the engine (when the partial engine load is low, the correction factors are higher)"""
        
        self.calculate_total_power_required() #You need the P_partial values
        
        #Import the correction factors table
        self.corf = pd.read_excel (r'correctionfactors.xlsx')
        
        for i in range(20):
        #If the partial engine load is smaller or equal to 5%, the correction factors corresponding to P_partial = 5% are assigned.
            if self.P_partial <= self.corf.iloc[0, 0]:
                self.corf_CO2 = self.corf.iloc[0, 5]
                self.corf_PM10 = self.corf.iloc[0, 6]
        
            #The NOX correction factors are dependend on the construction year of the engine and the weight class
                if self.c_year < 2008:
                    self.corf_NOX = self.corf.iloc[0, 1]   #<= CCR-1 class
                if 2008 <= self.c_year <= 2019:
                    self.corf_NOX = self.corf.iloc[0, 2]   #CCR-2 / Stage IIIa
                if self.c_year > 2019:
                    if self.L_w == 1:            #
                        self.corf_NOX = self.corf.iloc[0, 3]    #Stage V: IWP/IWA-v/c-3 class (vessels with P <300 kW: assumed to be weight class L1)
                    else:
                        self.corf_NOX = self.corf.iloc[0, 4]    #Stage V:IWP/IWA-v/c-4 class (vessels with P >300 kw: assumed to be weight class L2-L3)
        
            #If the partial engine load is greater than 5%:
            #It is determined inbetween which two percentages in the table the partial engine load lies
            #The correction factor is determined by means of linear interpolation
            
            elif self.corf.iloc[i, 0] < self.P_partial <= self.corf.iloc[i + 1, 0]:
                self.corf_CO2 = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 5] - self.corf.iloc[i, 5])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 5]
                self.corf_PM10 = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 6] - self.corf.iloc[i, 6])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 6]

                if self.c_year < 2008:
                    self.corf_NOX = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 1] - self.corf.iloc[i, 1])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 1]
                if 2008 <= self.c_year <= 2019:
                    self.corf_NOX = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 2] - self.corf.iloc[i, 2])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 2]
                if self.c_year > 2019:
                    if self.L_w ==1:
                        self.corf_NOX = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 3] - self.corf.iloc[i, 3])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 3]
                    else:
                        self.corf_NOX = ((self.P_partial - self.corf.iloc[i, 0]) * (self.corf.iloc[i+1, 4] - self.corf.iloc[i, 4])) / (self.corf.iloc[i+1, 0] - self.corf.iloc[i, 0]) + self.corf.iloc[i, 4]

                    
        print('Correction factor of CO2 is', self.corf_CO2)
        print('Correction factor of PM10 is', self.corf_PM10)
        print('Correction factor of NOX is', self.corf_NOX)

                    
    def calculate_emission_factors_total(self):
        """Calculating the total emission factors, by multiplying the general emission factors with the 
        correction factors"""
        
        print('The construction year of the engine is', self.c_year)   
        #self.calculate_engine_age() #You need the values of c_year
        
        self.emission_factors_general()  #You need the values of the general emission factors of CO2, PM10, NOX
        self.correction_factors() #You need the correction factors of CO2, PM10, NOX
        
        
        #The total emission factor is calculated by multiplying the general emission factor (EM_CO2 / EM_PM10 / EM_NOX)
        # By the correction factor (corf_CO2 / corf_PM10 / corf_NOX)
        
        self.Emf_CO2 = self.EM_CO2 * self.corf_CO2
        self.Emf_PM10 = self.EM_PM10 * self.corf_PM10
        self.Emf_NOX = self.EM_NOX * self.corf_NOX
        
        print('The total emission factor of CO2 is', self.Emf_CO2, 'g/kWh')
        print('The total emission factor of PM10 is', self.Emf_PM10, 'g/kWh')
        print('The total emission factor CO2 is', self.Emf_NOX, 'g/kWh')  

In [32]:
# Define a EnergyCalculation class (this class postprocesses all results and calculates the energy (kWh))
class EnergyCalculation:
    """
    Add information on energy use and effects on energy use.
    """

    def __init__(self, vessel, *args, **kwargs):
        super().__init__(*args, **kwargs)

        """Initialization"""
        self.vessel = vessel
        
        self.energy_use = {"time_start": [], 
                           "time_stop": [], 
                           "edge_start": [],
                           "edge_stop": [],
                           "total_energy": [], 
                           "total_emission_CO2": [],
                           "total_emission_PM10": [], 
                           "total_emission_NOX": [], 
                           "stationary": []}
        
        self.co2_footprint = {"total_footprint": 0, "stationary": 0}
        self.mki_footprint = {"total_footprint": 0, "stationary": 0}

    def calculate_energy_consumption(self):
        """Calculation of energy consumption based on total time in system and properties"""

        def calculate_distance(geom_start, geom_stop):
            """method to calculate the distance in meters between two geometries"""
            wgs84 = pyproj.Geod(ellps='WGS84')

            # distance between two points
            return float(wgs84.inv(geom_start.x, geom_start.y,
                                 geom_stop.x,  geom_stop.y) [2])
        
        # log messages that are related to locking
        stationary_phase_indicator = [
            "Waiting to enter waiting area stop",
            "Waiting in waiting area stop",
            "Waiting in line-up area stop",
            "Passing lock stop",
        ]
        
        # extract relevant elements from the vessel log
        times = self.vessel.log["Timestamp"]
        messages = self.vessel.log["Message"]
        geometries = self.vessel.log["Geometry"]

        # now walk past each logged event (each 'time interval' in the log corresponds to an event)
        for i in range(len(times) - 1):
            # determine the time associated with the logged event (how long did it last)
            delta_t = (times[i + 1] - times[i]).total_seconds()
            
            if delta_t != 0:
                # append time information to the variables for the dataframe
                self.energy_use["time_start"].append(times[i])
                self.energy_use["time_stop"].append(times[i + 1])
                
                # append geometry information to the variables for the dataframe
                self.energy_use["edge_start"].append(geometries[i])
                self.energy_use["edge_stop"].append(geometries[i + 1])
                
                #print(i)
                #print(FG_new.edges[('({:f}, {:f})'.format(geometries[i].x, geometries[i].y),'({:f}, {:f})'.format(geometries[i+1].x, geometries[i+1].y))])

                # calculate the distance travelled and the associated velocity 
                distance = calculate_distance(geometries[i], geometries[i + 1])
                V_0 = distance / delta_t
                
                # printstatements to check the output (can be removed later)
                print('delta_t: {:.4f} s'. format(delta_t))
                print('distance: {:.4f} m'. format(distance))
                print('velocity: {:.4f} m/s'. format(V_0))
                
                # we use the calculated velocity to determine the resistance (at h=10 for now) and power required
                self.vessel.calculate_total_resistance(V_0, h=10)
                self.vessel.calculate_total_power_required()
                
                self.vessel.calculate_emission_factors_total()

                if messages[i + 1] in stationary_phase_indicator:  # if we are in a stationary stage only log P_hotel
                    #Energy consumed per time step delta_t in the stationary stage
                    energy_delta = self.vessel.P_hotel * delta_t / 3600  # kJ/3600 = kWh
                    
                    #Emissions CO2, PM10 and NOX, in gram - emitted in the stationary stage per time step delta_t, consuming 'energy_delta' kWh
                    emission_delta_CO2 = self.vessel.Emf_CO2 * energy_delta # in g
                    emission_delta_PM10 = self.vessel.Emf_PM10 * energy_delta # in g
                    emission_delta_NOX = self.vessel.Emf_NOX * energy_delta # in g
                    
                    self.energy_use["total_energy"].append(energy_delta)
                    self.energy_use["stationary"].append(energy_delta)
                    self.energy_use["total_emission_CO2"].append(emission_delta_CO2)
                    self.energy_use["total_emission_PM10"].append(emission_delta_PM10)
                    self.energy_use["total_emission_NOX"].append(emission_delta_NOX)

                else:  # otherwise log P_tot
                    #Energy consumed per time step delta_t in the propulsion stage
                    energy_delta = self.vessel.P_tot * delta_t / 3600  # kJ/3600 = kWh
                   
                    #Emissions CO2, PM10 and NOX, in gram - emitted in the propulsion stage per time step delta_t, consuming 'energy_delta' kWh
                    emission_delta_CO2 = self.vessel.Emf_CO2 * energy_delta #Energy consumed per time step delta_t in the stationary phase # in g
                    emission_delta_PM10 = self.vessel.Emf_PM10 * energy_delta # in g
                    emission_delta_NOX = self.vessel.Emf_NOX * energy_delta # in g
    
                    self.energy_use["total_energy"].append(energy_delta)
                    self.energy_use["stationary"].append(0)
                    self.energy_use["total_emission_CO2"].append(emission_delta_CO2)
                    self.energy_use["total_emission_PM10"].append(emission_delta_PM10)
                    self.energy_use["total_emission_NOX"].append(emission_delta_NOX)
        
        # TODO: er moet hier een heel aantal dingen beter worden ingevuld
        # - de kruissnelheid is nu nog per default 1 m/s (zie de Movable mixin). Eigenlijk moet in de 
        #   vessel database ook nog een speed_loaded en een speed_unloaded worden toegevoegd. 
        # - er zou nog eens goed gekeken moeten worden wat er gedaan kan worden rond kustwerken 
        # - en er is nog iets mis met de snelheid rond een sluis


    def plot(self):
        
        import folium

        df = pd.DataFrame.from_dict(energycalculation.energy_use)

        m = folium.Map(location=[51.7, 4.4], zoom_start = 12)

        line = []
        for index, row in df.iterrows():
            line.append((row["edge_start"].y, row["edge_start"].x))

        folium.PolyLine(line, weight = 4).add_to(m)

        return m

### 1. Load Vaarweginformatie.nl graph

In [33]:
# create a cached version to speed up loading (remove cached file if a better yaml file is available)
fname = "fis_cache\\{}.pkl".format('FIS')

if os.path.exists(fname):
    print('I am loading cached network')
    with open(fname, 'rb') as pkl_file:
        FG = pickle.load(pkl_file)
        pkl_file.close()

else:
    print('I am getting new network')
    FG = load_fis_network(url)

    os.makedirs(os.path.dirname(fname), exist_ok=True)
    with open(fname, 'wb') as pkl_file:
        pickle.dump(FG, pkl_file)
        pkl_file.close()

I am loading cached network


In [34]:
# create a library of some interesting locations (lon, lat)
locations = {
    'Transferium Maasvlakte': shapely.geometry.Point(4.087406, 51.936737),
    'Neusse': shapely.geometry.Point(6.708892, 51.215737),
    'Basel': shapely.geometry.Point(7.640572, 47.555449),
    'Volkerak locks':  shapely.geometry.Point(4.4080241, 51.6897345),
    'Kreekrak locks':  shapely.geometry.Point(4.2304355, 51.4496009)
}

locations_nodes = {n: find_closest_node(FG, locations[n])[0] for n in locations}

In [35]:
locations_nodes

{'Transferium Maasvlakte': 22161408.0,
 'Neusse': 'FN410',
 'Basel': 'FN16',
 'Volkerak locks': 'L42863_B',
 'Kreekrak locks': 'B43908_A'}

### 2. Prepare vessel objects

#### 2.1 Load vessel database

In [36]:
# NB: This database needs to be triple checked (!!!)
# use the 'Vessel database v2.ipynb' notebook to generate 'Vessel-database-2.csv'
# for every value add a traceable source
vessel_db = pd.read_csv(location_vessel_database)
vessel_db.head()

,VesselID,B,L,H_e,H_f,T_e,T_f,emissionfactor,P_installed,L_w,...,capacity_loaded,capacity_unloaded,speed_loaded,speed_unloaded,resistance_loaded,resistance_unloaded,is_loaded,avv_class,cemt_class,type
0,29a24858-4a51-11e9-9792-b469212bff5b,5.06,39.76,2.0,5.25,1.2,2.5,0.73,175.0,1.0,...,260.0,0.0,3.083,4.470,16919.0,9188.0,0.0,M1,I,Spits
1,dcc05ff4-4972-11e9-a543-b469212bff5b,6.42,52.26,2.0,6.10,1.4,2.6,0.72,250.0,1.0,...,422.5,0.0,3.361,4.638,28770.0,17524.0,0.0,M2,II,Kempenaar
2,e6cea50a-4972-11e9-a1f1-b469212bff5b,7.15,72.21,2.0,6.40,1.5,2.6,0.72,435.0,1.0,...,520.0,0.0,3.861,4.638,43935.0,22746.0,0.0,M3,III,Hagenaar
3,f0eb9740-4972-11e9-96ab-b469212bff5b,8.02,67.02,2.0,6.60,1.5,2.7,0.72,435.0,1.0,...,682.5,0.0,4.000,4.889,50582.0,28777.0,0.0,M4,III,Dortmund-Eems (L <= 74 m)
4,f5685128-4972-11e9-9bc8-b469212bff5b,8.18,80.56,2.0,6.40,1.5,2.7,0.72,435.0,2.0,...,812.5,0.0,3.916,5.027,54096.0,31794.0,0.0,M5,III,Verl. Dortmund (L > 74 m)


In [37]:
# renaming some of the vessel_db fields to align them with the code (NB: this alignment should be improved)
# in the end vessel_db should hold all (and not more than) the fields required by Vessel (see further down) 

# rename fields
vessel_db = vessel_db.rename(columns={
                                      'capacity_loaded': 'capacity'}
                                     )

# remove parameters that are not used (later we may want to add these via a mix-in)
del vessel_db['VesselID']
del vessel_db['own_weight']
del vessel_db['capacity_unloaded']
del vessel_db['speed_loaded']
del vessel_db['speed_unloaded']
del vessel_db['is_loaded']
del vessel_db['avv_class']
del vessel_db['cemt_class']
del vessel_db['resistance_loaded']
del vessel_db['resistance_unloaded']
del vessel_db['emissionfactor']

In [38]:
# add fields that are necessary for the energy calculation (NB: now added blanket values, may need to be calculated)
vessel_db['C_b'] = 0.922218379546862
vessel_db['C_BB'] = 0.185

In [39]:
# a table with these fields is currently what the calculations need as input 
vessel_db.head()

,B,L,H_e,H_f,T_e,T_f,P_installed,L_w,capacity,type,C_b,C_BB
0,5.06,39.76,2.0,5.25,1.2,2.5,175.0,1.0,260.0,Spits,0.922218,0.185
1,6.42,52.26,2.0,6.10,1.4,2.6,250.0,1.0,422.5,Kempenaar,0.922218,0.185
2,7.15,72.21,2.0,6.40,1.5,2.6,435.0,1.0,520.0,Hagenaar,0.922218,0.185
3,8.02,67.02,2.0,6.60,1.5,2.7,435.0,1.0,682.5,Dortmund-Eems (L <= 74 m),0.922218,0.185
4,8.18,80.56,2.0,6.40,1.5,2.7,435.0,2.0,812.5,Verl. Dortmund (L > 74 m),0.922218,0.185


#### 2.2 Create a vessel object with the new mixins, and use this in the vessel generator

In [40]:
# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (opentnsim.core.Identifiable, opentnsim.core.ContainerDependentMovable, 
                          opentnsim.core.HasResource, opentnsim.core.Routeable, 
                          VesselProperties, ConsumesEnergy), {})

# For testing purposes we only want v to be 1
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

# identify which vessel type to extract from the vessel database (change the name to change the vessel used)
vessel_type = 'Dortmund-Eems (L <= 74 m)' # <==============

# prepare data for the vessel
data_vessel = {"env": None,
               "name": "Vessel number 1",
               "route": None,
               "geometry": None,  # lon, lat
               "capacity": vessel_db[vessel_db['type']==vessel_type]['capacity'].values[0],
               "compute_v": compute_v_provider(v_empty=1, v_full=1),
               "type": vessel_db[vessel_db['type']==vessel_type]['type'].values[0],
               "B": vessel_db[vessel_db['type']==vessel_type]['B'].values[0],
               "L": vessel_db[vessel_db['type']==vessel_type]['L'].values[0],
               "H_e": vessel_db[vessel_db['type']==vessel_type]['H_e'].values[0],
               "H_f": vessel_db[vessel_db['type']==vessel_type]['H_f'].values[0],
               "T_e": vessel_db[vessel_db['type']==vessel_type]['T_e'].values[0],
               "T_f": vessel_db[vessel_db['type']==vessel_type]['T_f'].values[0],
               "P_installed": vessel_db[vessel_db['type']==vessel_type]['P_installed'].values[0],
               "L_w": vessel_db[vessel_db['type']==vessel_type]['L_w'].values[0],
               "C_b": vessel_db[vessel_db['type']==vessel_type]['C_b'].values[0],
               "C_BB": vessel_db[vessel_db['type']==vessel_type]['C_BB'].values[0],
              }

print(data_vessel)

# create the transport processing resource
vessels = []
vessel = TransportResource(**data_vessel)

{'env': None, 'name': 'Vessel number 1', 'route': None, 'geometry': None, 'capacity': 682.5, 'compute_v': <function compute_v_provider.<locals>.<lambda> at 0x000001E56846EB80>, 'type': 'Dortmund-Eems (L <= 74 m)', 'B': 8.02, 'L': 67.02, 'H_e': 2.0, 'H_f': 6.6, 'T_e': 1.5, 'T_f': 2.7, 'P_installed': 435.0, 'L_w': 1.0, 'C_b': 0.922218379546862, 'C_BB': 0.185}
The construction year of the engine is 1997


### 3. Run simulation
This example now sends 1 vessel over the network 

To do: 
 - extend with multiple vessels
 - check energy consumption calculation

In [41]:
# Start simpy environment (at a given date and time) (simulation_start = datetime.datetime.now() to just start now)
simulation_start = datetime.datetime(2020, 9, 26) # <============ start time
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.epoch = time.mktime(simulation_start.timetuple())

# Add graph to environment
env.FG = FG

In [42]:
# Add environment, route/path and startlocation to the vessel
start_point = locations_nodes['Volkerak locks'] # <============== start point
end_point = locations_nodes['Kreekrak locks']   # <============== end point
path = nx.dijkstra_path(FG, start_point, end_point, weight='Length') # NB: dijkstra based on shortest route

vessel.env = env
vessel.route = path
vessel.geometry = FG.nodes[start_point]['geometry'] # <========== vessel start point

route = nx.DiGraph(FG.subgraph(path))

In [43]:
def start(env, vessel):
    """The start process moves a vessel over the network 
    (from its 'vessel.geometry' position to the end of 'vessel.route')"""
    vessel.log_entry("Start sailing", env.now, "", vessel.geometry)
    yield from vessel.move()
    vessel.log_entry("Stop sailing", env.now, "", vessel.geometry)
        
env.process(start(env, vessel))
env.run()

### 5. Calculate energy use

In [44]:
# basic inspection of the vessel log
pd.DataFrame.from_dict(vessel.log).head(5)

,Message,Timestamp,Value,Geometry
0,Start sailing,2020-09-26 00:00:00.000000,,POINT (4.40878376619477 51.6893044653478)
1,Sailing from node L42863_B to node B34113_A su...,2020-09-26 00:00:00.000000,0,POINT (4.40878376619477 51.6893044653478)
2,Sailing from node L42863_B to node B34113_A su...,2020-09-26 00:01:37.988698,0,POINT (4.407711423268813 51.68872877120878)
3,Sailing from node B34113_A to node B34113_B su...,2020-09-26 00:01:37.988698,0,POINT (4.407711423268813 51.68872877120878)
4,Sailing from node B34113_A to node B34113_B su...,2020-09-26 00:01:46.039699,0,POINT (4.407623317192122 51.68868147089958)


In [45]:
# create an EnergyCalculation object (vessel is used to initialise)
energycalculation = EnergyCalculation(vessel)

In [46]:
# apply EnergyCalculation method to post process vessel.log
energycalculation.calculate_energy_consumption()

# for now intermediate output is generated to show that the results are logical

delta_t: 97.9887 s
distance: 97.9887 m
velocity: 1.0000 m/s
The total power required is 37.79209773808974 kW
The partial load is 0.08687838560480399 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The general emission factor of PM10 is 0.3 g/kWh
The general emission factor CO2 is 9.4 g/kWh
The total power required is 37.79209773808974 kW
The partial load is 0.08687838560480399 kW
Correction factor of CO2 is 1.2204972915161567
Correction factor of PM10 is 1.8425701532021754
Correction factor of NOX is 1.468591821072921
The total emission factor of CO2 is 793.3232394855019 g/kWh
The total emission factor of PM10 is 0.5527710459606526 g/kWh
The total emission factor CO2 is 13.804763118085457 g/kWh
delta_t: 8.0510 s
distance: 8.0510 m
velocity: 1.0000 m/s
The total power required is 37.7920973114678 kW
The partial load is 0.0868783846240639 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The gener

Correction factor of CO2 is 1.2204972916100498
Correction factor of PM10 is 1.8425701551035096
Correction factor of NOX is 1.468591822223111
The total emission factor of CO2 is 793.3232395465324 g/kWh
The total emission factor of PM10 is 0.5527710465310529 g/kWh
The total emission factor CO2 is 13.804763128897243 g/kWh
delta_t: 897.5273 s
distance: 897.5273 m
velocity: 1.0000 m/s
The total power required is 37.7920977011201 kW
The partial load is 0.08687838551981632 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The general emission factor of PM10 is 0.3 g/kWh
The general emission factor CO2 is 9.4 g/kWh
The total power required is 37.7920977011201 kW
The partial load is 0.08687838551981632 kW
Correction factor of CO2 is 1.220497291584147
Correction factor of PM10 is 1.8425701545789757
Correction factor of NOX is 1.4685918219058003
The total emission factor of CO2 is 793.3232395296956 g/kWh
The total emission factor of PM10 is 0.552771046

Correction factor of CO2 is 1.2204972915851364
Correction factor of PM10 is 1.8425701545990132
Correction factor of NOX is 1.4685918219179217
The total emission factor of CO2 is 793.3232395303387 g/kWh
The total emission factor of PM10 is 0.552771046379704 g/kWh
The total emission factor CO2 is 13.804763126028465 g/kWh
delta_t: 456.3306 s
distance: 456.3306 m
velocity: 1.0000 m/s
The total power required is 37.7920976939352 kW
The partial load is 0.08687838550329931 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The general emission factor of PM10 is 0.3 g/kWh
The general emission factor CO2 is 9.4 g/kWh
The total power required is 37.7920976939352 kW
The partial load is 0.08687838550329931 kW
Correction factor of CO2 is 1.2204972915973604
Correction factor of PM10 is 1.8425701548465512
Correction factor of NOX is 1.4685918220676668
The total emission factor of CO2 is 793.3232395382843 g/kWh
The total emission factor of PM10 is 0.55277104

Correction factor of CO2 is 1.2204972915910375
Correction factor of PM10 is 1.8425701547185074
Correction factor of NOX is 1.4685918219902083
The total emission factor of CO2 is 793.3232395341744 g/kWh
The total emission factor of PM10 is 0.5527710464155522 g/kWh
The total emission factor CO2 is 13.80476312670796 g/kWh
delta_t: 669.9727 s
distance: 669.9727 m
velocity: 1.0000 m/s
The total power required is 37.79209769693195 kW
The partial load is 0.08687838551018838 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The general emission factor of PM10 is 0.3 g/kWh
The general emission factor CO2 is 9.4 g/kWh
The total power required is 37.79209769693195 kW
The partial load is 0.08687838551018838 kW
Correction factor of CO2 is 1.2204972915918493
Correction factor of PM10 is 1.8425701547349482
Correction factor of NOX is 1.468591822000154
The total emission factor of CO2 is 793.323239534702 g/kWh
The total emission factor of PM10 is 0.55277104

The total emission factor of PM10 is 0.5527710463690316 g/kWh
The total emission factor CO2 is 13.804763125826174 g/kWh
delta_t: 103.0463 s
distance: 103.0463 m
velocity: 1.0000 m/s
The total power required is 37.79209768699683 kW
The partial load is 0.08687838548734904 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The general emission factor of PM10 is 0.3 g/kWh
The general emission factor CO2 is 9.4 g/kWh
The total power required is 37.79209768699683 kW
The partial load is 0.08687838548734904 kW
Correction factor of CO2 is 1.2204972916101207
Correction factor of PM10 is 1.8425701551049456
Correction factor of NOX is 1.4685918222239795
The total emission factor of CO2 is 793.3232395465784 g/kWh
The total emission factor of PM10 is 0.5527710465314837 g/kWh
The total emission factor CO2 is 13.804763128905408 g/kWh
delta_t: 564.0601 s
distance: 564.0601 m
velocity: 1.0000 m/s
The total power required is 37.792097697612114 kW
The partial lo

...

delta_t: 460.6509 s
distance: 460.6509 m
velocity: 1.0000 m/s
The total power required is 37.7920976962143 kW
The partial load is 0.08687838550853862 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The general emission factor of PM10 is 0.3 g/kWh
The general emission factor CO2 is 9.4 g/kWh
The total power required is 37.7920976962143 kW
The partial load is 0.08687838550853862 kW
Correction factor of CO2 is 1.220497291593169
Correction factor of PM10 is 1.8425701547616744
Correction factor of NOX is 1.4685918220163217
The total emission factor of CO2 is 793.3232395355599 g/kWh
The total emission factor of PM10 is 0.5527710464285023 g/kWh
The total emission factor CO2 is 13.804763126953423 g/kWh
delta_t: 1076.8132 s
distance: 1076.8132 m
velocity: 1.0000 m/s
The total power required is 37.79209769690923 kW
The partial load is 0.08687838551013616 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
T

Correction factor of CO2 is 1.220497291589374
Correction factor of PM10 is 1.842570154684823
Correction factor of NOX is 1.4685918219698313
The total emission factor of CO2 is 793.3232395330931 g/kWh
The total emission factor of PM10 is 0.5527710464054468 g/kWh
The total emission factor CO2 is 13.804763126516415 g/kWh
delta_t: 37.6432 s
distance: 37.6432 m
velocity: 1.0000 m/s
The total power required is 37.79209765846848 kW
The partial load is 0.08687838542176661 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The general emission factor of PM10 is 0.3 g/kWh
The general emission factor CO2 is 9.4 g/kWh
The total power required is 37.79209765846848 kW
The partial load is 0.08687838542176661 kW
Correction factor of CO2 is 1.2204972916625867
Correction factor of PM10 is 1.842570156167381
Correction factor of NOX is 1.4685918228666872
The total emission factor of CO2 is 793.3232395806814 g/kWh
The total emission factor of PM10 is 0.5527710468

Correction factor of CO2 is 1.2204972916000474
Correction factor of PM10 is 1.8425701549009612
Correction factor of NOX is 1.4685918221005816
The total emission factor of CO2 is 793.3232395400308 g/kWh
The total emission factor of PM10 is 0.5527710464702883 g/kWh
The total emission factor CO2 is 13.804763127745467 g/kWh
delta_t: 490.3753 s
distance: 490.3753 m
velocity: 1.0000 m/s
The total power required is 37.792097690001775 kW
The partial load is 0.08687838549425696 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The general emission factor of PM10 is 0.3 g/kWh
The general emission factor CO2 is 9.4 g/kWh
The total power required is 37.792097690001775 kW
The partial load is 0.08687838549425696 kW
Correction factor of CO2 is 1.2204972916045944
Correction factor of PM10 is 1.8425701549930373
Correction factor of NOX is 1.468591822156282
The total emission factor of CO2 is 793.3232395429864 g/kWh
The total emission factor of PM10 is 0.5527

Correction factor of NOX is 1.4685918219793739
The total emission factor of CO2 is 793.3232395335993 g/kWh
The total emission factor of PM10 is 0.5527710464101793 g/kWh
The total emission factor CO2 is 13.804763126606115 g/kWh
delta_t: 628.3906 s
distance: 628.3906 m
velocity: 1.0000 m/s
The total power required is 37.79209769052956 kW
The partial load is 0.08687838549547025 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The general emission factor of PM10 is 0.3 g/kWh
The general emission factor CO2 is 9.4 g/kWh
The total power required is 37.79209769052956 kW
The partial load is 0.08687838549547025 kW
Correction factor of CO2 is 1.2204972916036239
Correction factor of PM10 is 1.842570154973382
Correction factor of NOX is 1.4685918221443917
The total emission factor of CO2 is 793.3232395423555 g/kWh
The total emission factor of PM10 is 0.5527710464920146 g/kWh
The total emission factor CO2 is 13.804763128157282 g/kWh
delta_t: 478.9061 s


The total power required is 37.792097691077714 kW
The partial load is 0.08687838549673038 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The general emission factor of PM10 is 0.3 g/kWh
The general emission factor CO2 is 9.4 g/kWh
The total power required is 37.792097691077714 kW
The partial load is 0.08687838549673038 kW
Correction factor of CO2 is 1.2204972916026158
Correction factor of PM10 is 1.8425701549529678
Correction factor of NOX is 1.4685918221320424
The total emission factor of CO2 is 793.3232395417002 g/kWh
The total emission factor of PM10 is 0.5527710464858904 g/kWh
The total emission factor CO2 is 13.804763128041198 g/kWh
delta_t: 691.3476 s
distance: 691.3476 m
velocity: 1.0000 m/s
The total power required is 37.79209769889022 kW
The partial load is 0.08687838551469015 kW
The construction year of the engine is 1997
The general emission factor of CO2 is 650 g/kWh
The general emission factor of PM10 is 0.3 g/kWh
The general

In [47]:
pd.read_excel (r'Correctionfactors.xlsx')

,% Partial engine load,Nox <= CCR-1,Nox CCR-2 / Stage IIIa,Nox IWP/IWA-v/c-3,Nox IWP/IWA-v/c-4,CO2,PM10
0,0.05,1.83,2.02,3.99,4.79,1.25,2.44
1,0.10,1.34,1.42,2.63,3.07,1.21,1.63
2,0.15,1.17,1.27,2.12,2.42,1.18,1.32
3,0.20,1.10,1.19,1.85,2.08,1.15,1.19
4,0.25,1.06,1.15,1.69,1.88,1.13,1.12
5,0.30,1.04,1.13,1.58,1.73,1.11,1.08
6,0.35,1.03,1.11,1.50,1.63,1.09,1.05
7,0.40,1.02,1.09,1.44,1.56,1.07,1.03
8,0.45,1.01,1.08,1.39,1.50,1.05,1.01
9,0.50,1.00,1.07,1.35,1.45,1.04,1.01


In [48]:
df = pd.DataFrame.from_dict(energycalculation.energy_use)
df.head(5)

#Time is seconds
#Energy in kWh
#Emissions in gram

,time_start,time_stop,edge_start,edge_stop,total_energy,total_emission_CO2,total_emission_PM10,total_emission_NOX,stationary
0,2020-09-26 00:00:00.000000,2020-09-26 00:01:37.988698,POINT (4.40878376619477 51.6893044653478),POINT (4.407711423268813 51.68872877120878),1.028666,816.064831,0.568617,14.200494,0
1,2020-09-26 00:01:37.988698,2020-09-26 00:01:46.039699,POINT (4.407711423268813 51.68872877120878),POINT (4.407623317192122 51.68868147089958),0.084518,67.049964,0.046719,1.166749,0
2,2020-09-26 00:01:46.039699,2020-09-26 00:04:00.501781,POINT (4.407623317192122 51.68868147089958),POINT (4.40615183993005 51.6878914989466),1.411557,1119.820739,0.780268,19.486206,0
3,2020-09-26 00:04:00.501781,2020-09-26 00:23:56.405478,POINT (4.40615183993005 51.6878914989466),POINT (4.3925552967856 51.6812504548851),12.554364,9959.668488,6.939689,173.310017,0
4,2020-09-26 00:23:56.405478,2020-09-26 00:24:52.273815,POINT (4.3925552967856 51.6812504548851),POINT (4.39198470467551 51.6808950327794),0.586495,465.280037,0.324197,8.096423,0


In [49]:
np.sum(df['total_energy'])

702.2919534857617

In [50]:
# plot route used for the energy calculation
energycalculation.plot()

# Todo: check why Schelde Rijn canal is not used

In [ ]:
# Plot entire network
m = folium.Map(location=[51.7, 4.4], zoom_start = 10)

for edge in FG.edges(data = True):
    # print(edge)
    points_x = list(edge[2]["geometry"].coords.xy[0])
    points_y = list(edge[2]["geometry"].coords.xy[1])
    
    line = []
    for i, _ in enumerate(points_x):
        line.append((points_y[i], points_x[i]))
    
    if edge[2]["Name"] in ["Voorhavens Jachtensluis", "Voorhavens Volkeraksluizen"]:
        folium.PolyLine(line, color = "red", weight = 5, popup = edge[2]["Name"]).add_to(m)
    
    else:
        folium.PolyLine(line, weight = 2, popup = edge[2]["Name"]).add_to(m)

m

### Visualise on Google Earth

In [ ]:
plot.vessel_kml(env, [vessel], fname='vessel_movements.kml')

### Sandbox

### Create locks (locks not implemented yet in this example)
We can see on the maps that there are locks on the graph, but that the information on the locks is limited. The following edges represent locks (NB: click the edges to show info:

- Voorhavens Jachtensluis
- Voorhavens Volkeraksluizen

These edges will be replaced by two lock elements. The Jachtensluizen are mainly designed for yachts and pleasure craft, and have a length of 135 meters and a width of 16 meters. The Volkeraksluizen have three chambers each with a length of 330 meters and a width of 24 meters. For now we'll assume that both locks have a maximum allowable draught of 4.5 meters.

Additional information on the locks can be found on the [Rijkswaterstand website (link in Dutch)](https://www.rijkswaterstaat.nl/water/waterbeheer/bescherming-tegen-het-water/waterkeringen/deltawerken/volkeraksluizen.aspx). 

In [ ]:
lock_nr_1 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 1", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_2 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 2", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_3 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 3", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

# lock_test = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Jachtensluis", 
#                         node_1 = "(4.395179, 51.691512)", node_2 = "(4.408442, 51.700226)",
#                         lock_length = 330, lock_width = 24, lock_depth = 4.5, 
#                         doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)


In [ ]:
for edge in FG_new.edges(data = True):
    if edge[2]["Info"]["VWG_NAAM"] == "Voorhavens Volkeraksluizen":
        # For testing, all locks have the water level at the right side
        lock_nr_1.water_level = "(4.430289, 51.700047)"
        lock_nr_2.water_level = "(4.430289, 51.700047)"
        lock_nr_3.water_level = "(4.430289, 51.700047)"
        
        # Add locks to the correct edge
        FG_new.edges[edge[0], edge[1]]["Lock"] = [lock_nr_1, lock_nr_2, lock_nr_3]

### Show loginfo of the locks
The locking does not work yet in this example

In [ ]:
pd.DataFrame.from_dict(lock_nr_1.log)

In [ ]:
pd.DataFrame.from_dict(lock_nr_2.log)

In [ ]:
pd.DataFrame.from_dict(lock_nr_3.log)

In [ ]:
1